In [1]:
import pandas
import numpy as np
import cv2
from scipy.stats import norm
import os
from os import listdir
from os.path import isfile, join
import matplotlib.mlab as mlab
import math
import xlsxwriter
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from scipy.stats import norm
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D
from scipy import ndimage

#importing pandas as pd
import pandas as pd
from numpy import genfromtxt
from io import StringIO
import csv

In [2]:
def get_largest_component( msk ) : 
    n_components , labels , stats , _ = cv2.connectedComponentsWithStats( msk.astype( np.uint8 ) , connectivity = 4) 
    max_area = 0
    for i in range( 1 , n_components ) : 
        area = stats[ i , cv2.CC_STAT_AREA ] 
        if area > max_area:
            max_area = area
            msk = labels == i
    return msk
#crear una máscara 
def get_reds2(img) : 
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    msk = np.bitwise_or(img[ :  , :  , 2] < 0 , img[ :  , :  , 2] > 0)
    # usamos el valor y la imagen original
    img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB)
    return msk
#convierte en negro lo que no nos interesa en la máscara
def blackout( img , msk ) : 
    hsv = cv2.cvtColor( img , cv2.COLOR_RGB2HSV ) 
    h , s , v = cv2.split( hsv ) 
    v[ msk ] = 0
    hsv = cv2.merge(( h , s , v ) ) 
    img = cv2.cvtColor( hsv , cv2.COLOR_HSV2RGB ) 
    return img
#recibe la carpeta y el nombre del archivo que deseamos segmentar
#guarda la imagen y la regresa
def mascaraa(path1,path2,name):
    
    img = cv2.imread(join(path1,name))
    img2 = np.copy(img)
    
    nrows,ncols,_=img.shape
    Ximg = np.reshape(img,(nrows*ncols,3))
    
    y = model.predict(Ximg)
    yimg = np.reshape(y,(nrows,ncols))
    
    imgcarne = img.copy()
    imgcarne[yimg==0,:] = 0
    img = imgcarne
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    '''
    plt.figure()
    plt.imshow(img)
    plt.show()
    '''
    mask =  get_reds2(img)
    global new
    new = get_largest_component(mask)
    new = ndimage.binary_fill_holes(new).astype(int)
    new[new != 0] = 255
    res = new < 255
    x,y = np.where(res)
    '''
    plt.figure()
    plt.imshow(new,cmap="gray")
    plt.show()
    '''
    #x,y = np.where(x)
    #print(x.shape,y.shape)
    
    img2[x,y] = 0
    '''
    plt.figure()
    plt.imshow(img2)
    plt.show()
    '''
    
    os.chdir(path2)
    cv2.imwrite(name[0:-3] + "PNG", img2)
    return img2

# guarda la segmentación
#en una carpeta llamada segmentacion
#recibe a fila del excel que se desea segmentar y el tipo de
#corte que es
def pormuestra(arr,t,path1,path2):#t es si es cabina o afuera
    res = []
    for i in range(len(arr) - 3):
        aux = arr[1];
        aux = '0'+aux;
        ayuda = int(float(arr[3+i]))
        ayuda = str(ayuda)
        aux = ayuda+'_'+arr[0]+aux+'_'+arr[2]+'_'+t+'.JPG';
        res.append(aux)
    
    #print(res)
    #imgs = []
    for nombre in res:
        print("Procesando:",nombre)
        img = mascaraa(path1,path2,nombre)
        

# Entrenamiento SVC

In [3]:
carne = cv2.imread('fotos_para_entrenar/carne.JPG')
fondo = cv2.imread('fotos_para_entrenar/fondo.JPG') #fotos de la carne y el fondo

nrows,ncols,_= carne.shape
Xcarne = np.reshape(carne,(nrows*ncols,3))
nrows,ncols,_= fondo.shape
Xfondo = np.reshape(fondo,(nrows*ncols,3))#reshape

idx = np.random.permutation(nrows*ncols)[:5000]
Xfondo = Xfondo[idx,:]
Xcarne = Xcarne[idx,:]

X = np.concatenate((Xcarne,Xfondo),axis=0)
y = np.zeros(10000) #para distiguir cual es fondo y cual carne
y[:5000] = 1

model = SVC()
model.fit(X,y)#entrenamiento

SVC()

In [4]:

#os.mkdir("segmentadas")

# Para cabina

In [ ]:
#Leo los excels, los convierto a csv y genero
#un arreglo a partir de eso
read_file = pd.read_excel ("cabina.xlsx")
  
read_file.to_csv ("cabina.csv", index = None,header=True)


csvfile = open('cabina.csv', 'r')
reader = csv.reader(csvfile, delimiter=',')
my_list = list(reader)

my_list = np.array(my_list)

my_list[0][1] = my_list[0][1][1:]
for i in range(my_list.shape[0]):
    
    pormuestra(my_list[i],'c',r'C:\Users\PC Gamer\Desktop\Up7mo\Carnes\fotos_arrachera',r'C:\Users\PC Gamer\Desktop\Up7mo\Carnes\segmentadas')#'c' porque es en cabina
    

Procesando: 20210830_A01_1_c.JPG
Procesando: 20210901_A01_1_c.JPG


# Para afuera

In [ ]:
#Leo los excels, los convierto a csv y genero
#un arreglo a partir de eso
os.chdir(r'C:\Users\PC Gamer\Desktop\Up7mo\Carnes')
read_file = pd.read_excel ("afuera.xlsx")
  
read_file.to_csv ("afuera.csv", index = None,header=True)


csvfile = open('afuera.csv', 'r')
reader = csv.reader(csvfile, delimiter=',')
my_list = list(reader)

my_list = np.array(my_list)

my_list[0][1] = my_list[0][1][1:]
print

for i in range(my_list.shape[0]):
    
    pormuestra(my_list[i],'a',r'C:\Users\PC Gamer\Desktop\Up7mo\Carnes\fotos_sirloin',r'C:\Users\PC Gamer\Desktop\Up7mo\Carnes\segmentadas')

In [ ]:
plt.figure()
plt.imshow(new,cmap="gray")
plt.show()
Newnew = np.copy(new)
img = cv2.imread(r"C:\Users\PC Gamer\Desktop\Up7mo\Carnes\20210830_A01_1_c.JPG")
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(img)
plt.show()

In [ ]:
print(Newnew.shape)
print(Newnew[3864//2,5152//2])

In [ ]:
res = new < 255
x,y = np.where(res)
plt.figure()
plt.imshow(new)
plt.show()
#x,y = np.where(x)
#print(x.shape,y.shape)
img2 = np.copy(img)
img2[x,y] = 0
plt.figure()
plt.imshow(img2)
plt.show()